### 0. Libraries

In [14]:
import requests
import geopandas as gpd
import pandas as pd 
from shapely.geometry import Polygon

In [1]:
# RUN THIS IN THE COMMAND PROMPT: java -jar "D:\Climate_Shelter_Index\isocrona\graphhopper-web-7.0.jar" server "D:\Climate_Shelter_Index\isocrona\bologna.yml"

### 1. Isochrones
Calculate isochrones for each entrance to the green areas

In [4]:
# Check if GraphHopper is running

# Replace with the actual URL of your GraphHopper server
graphhopper_url = "http://localhost:8989"  # Change the URL as needed

try:
    # Send a GET request to the server's root URL
    response = requests.get(graphhopper_url)

    # Check the status code of the response
    if response.status_code == 200:
        print("GraphHopper is up and running.")
    else:
        print(f"GraphHopper returned a status code {response.status_code}.")
except requests.ConnectionError:
    print("Unable to connect to GraphHopper. Please check if it's running.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

GraphHopper is up and running.


In [9]:
# Set the URL of the GraphHopper server
graphhopper_url = "http://localhost:8989/isochrone"

# Load the shapefile containing points (entrances of green areas)
points_shapefile = "D:/Climate_Shelter_Index/aree-verdi_entrate_centroidi/aree-verdi_entrate_centroidi.shp"
# Load the shapefile as a GeoDataFrame
points_gdf = gpd.read_file(points_shapefile)

In [15]:
# Initialize an empty GeoDataFrame to store isochrone results
isochrone_gdf = gpd.GeoDataFrame(columns=["green_area_name", "minutes_distance", "geometry"], crs="epsg:4326")

# Iterate through each point in the GeoDataFrame
for index, point in points_gdf.iterrows():
    # Extract latitude and longitude from the point geometry
    lat, lon = point.geometry.y, point.geometry.x

    # Define the isochrone query parameters
    minutes_distance = 5
    timeseconds = str(minutes_distance * 60)
    query = {
        "point": f"{lat},{lon}",
        "time_limit": timeseconds,
        "distance_limit": "0",
        "profile": "foot",
        "buckets": "1",
        "reverse_flow": "false"
    }

    # Send a GET request to the GraphHopper server
    response = requests.get(graphhopper_url, params=query)
    data = response.json()

    # Extract the green area name from the 'nome' column
    green_area_name = point['nome']

    # Extract the isochrone geometry and create a Polygon
    geotype = data['polygons'][0]['geometry']['type']
    geometry = None
    if geotype == "Polygon":
        geometry = Polygon(data['polygons'][0]['geometry']['coordinates'][0])

    # Create a temporary DataFrame
    temp_df = pd.DataFrame({
        "green_area_name": [green_area_name],
        "minutes_distance": [minutes_distance],
        "geometry": [geometry]
    })

    # Append the temporary DataFrame to the isochrone GeoDataFrame
    isochrone_gdf = pd.concat([isochrone_gdf, temp_df], ignore_index=True)

In [26]:
# Convert the concatenated DataFrame back to a GeoDataFrame
isochrone_gdf = gpd.GeoDataFrame(isochrone_gdf)

In [27]:
# Set the CRS for the isochrone GeoDataFrame to EPSG 4326
isochrone_gdf.crs = "epsg:4326"


In [31]:
# Rename columns
isochrone_gdf = isochrone_gdf.rename(columns= {'green_area_name':'ga_name',
                                               'minutes_distance': 'min_dist' })

In [32]:
isochrone_gdf.head()

,ga_name,min_dist,geometry
0,NaN,5,"POLYGON ((11.28736 44.49106, 11.28736 44.49101..."
1,GIARDINO DEL GHISELLO (EX GIARDINO CALDA),5,"POLYGON ((11.29939 44.49114, 11.29976 44.49128..."
2,PARCO DI VILLA GROSSO,5,"POLYGON ((11.33795 44.52134, 11.33776 44.52101..."
3,NUOVA,5,"POLYGON ((11.35273 44.48368, 11.35279 44.48339..."
4,AREA VIA DELL'ANGELO CUSTODE,5,"POLYGON ((11.36123 44.45897, 11.36461 44.45888..."


In [33]:
# Save the isochrone results to a new shapefile
output_shapefile = "D:/Climate_Shelter_Index/isocrona/5min_isochrones.shp"  
isochrone_gdf.to_file(output_shapefile)